In [9]:
from random import random, randrange
import cv2 as cv
import numpy as np
import imutils
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import math

In [10]:
corner1 = [1400,1300]
corner2 = [1650,1075]
robotWidth = 50
commands = []
speed = 20
curveSpeed = 10
angle = 90
sideAngle = 90
target = 'target'
offsetX = 0
offsetY = 0
imageAngle = 8.8
resolution = 0.02
offset = 0
# robotLength = 26
robotLength = 0
# addOffsetX = corner1[0]
# addOffsetY = areaHeight-corner1[1]+selectHeight
inflated = cv.imread("updatedImage.png")
size = inflated.shape
areaWidth = size[1]
areaHeight = size[0]

def rotate(angle, pixelsX, pixelsY, offsetX, offsetY):
    x = (pixelsX)*math.cos(math.radians(angle)) - (-pixelsY)*math.sin(math.radians(angle)) + offsetX/resolution
    y = (pixelsY)*math.cos(math.radians(angle)) - (pixelsX)*math.sin(math.radians(angle)) + offsetY/resolution
    pixels = [x,y]
    return pixels

def pixelsToCoordinates(pixelsX, pixelsY, resolution):
    x = pixelsX*resolution
    y = pixelsY*resolution
    coordinates = [x,y]
    return coordinates

def prepareImage(image, angle):
    image = cv.imread(image)
    rotatedOriginal = imutils.rotate_bound(image,angle)
    cv.imwrite("rotated.png",rotatedOriginal)
    rotated = cv.imread("rotated.png")
    retVal,masked_image = cv.threshold(rotated,200,255,cv.THRESH_BINARY)
    kernel = np.ones((23,23),np.uint8)
    inflated = cv.erode(masked_image, kernel, iterations=1)
    return inflated

def calculateOffset(image,angle):
    size = image.shape
    height = 0
    width = size[1]
    print(width)
    # offset = width*math.cos(math.radians(angle)) + height*math.sin(math.radians(angle))
    # y = (pixelsY)*math.cos(math.radians(angle)) - (pixelsX)*math.sin(math.radians(angle))
    offset = height*math.cos(math.radians(angle)) - width*math.sin(math.radians(angle))
    offset = abs(offset*resolution)
    return offset

def getWhitePixels(image, originalImage, corner1, corner2, markedPixels):
    whitePixels = np.argwhere(image == 255)
    whitePixelsOriginal = np.argwhere(originalImage == 255)
    whitePixelsArr = []
    whitePixelsOriginalArr = []
    for i in range(len(whitePixelsOriginal)):
        if whitePixelsOriginal[i][1] >= corner1[0]+int(robotWidth/2.0) and whitePixelsOriginal[i][1] <= corner2[0]-int(robotWidth/2.0) and whitePixelsOriginal[i][0] >= corner2[1]+int(robotWidth/2.0) and whitePixelsOriginal[i][0] <= corner1[1]-int(robotWidth/2.0):
            if(whitePixelsOriginal[i][2] == 0):
                whitePixelsOriginalArr.append([whitePixelsOriginal[i][1],whitePixelsOriginal[i][0]])
    for i in range(len(whitePixels)):
        if whitePixels[i][1] >= corner1[0]+int(robotWidth/2.0) and whitePixels[i][1] <= corner2[0]-int(robotWidth/2.0) and whitePixels[i][0] >= corner2[1]+int(robotWidth/2.0) and whitePixels[i][0] <= corner1[1]-int(robotWidth/2.0):
            if(whitePixels[i][2] == 0):
                whitePixelsArr.append([whitePixels[i][1],whitePixels[i][0]])
    for i in range(len(whitePixelsOriginalArr)):
        if whitePixelsOriginalArr[i] not in whitePixelsArr and whitePixelsOriginalArr[i] not in markedPixels:
            markedPixels.append(whitePixelsOriginalArr[i])

    # for i in range(len(whitePixelsArr)):
    return whitePixelsArr,markedPixels

def BezierPoints(start,middle,end):
    p0 = np.array([start[0], start[1]])
    p1 = np.array([middle[0], middle[1]])
    p2 = np.array([end[0], end[1]])
    return p0,p1,p2

def B(t,p0,p1,p2):
    return (1-t)**2*p0 + 2*t*(1-t)*p1 + t**2*p2

def getMaxLineLength(whitePixelsArr,robotWidth,finish,direction):
    length = 0
    if direction == 'up':
        for j in range(len(whitePixelsArr)):
            if(whitePixelsArr[j][0] == finish[0]):
                if(abs(finish[1] - whitePixelsArr[j][1]) - int(robotWidth/2.0) > length):
                    length = abs(finish[1] - whitePixelsArr[j][1]) - int(robotWidth/2.0)
    elif direction == 'left':
        for j in range(len(whitePixelsArr)):
            if(whitePixelsArr[j][1] == finish[1]):
                if(abs(finish[0] - whitePixelsArr[j][0]) - int(robotWidth/2.0) > length):
                    length = abs(finish[0] - whitePixelsArr[j][0]) - int(robotWidth/2.0)
    elif direction == 'right':
        for j in range(len(whitePixelsArr)):
            if(whitePixelsArr[j][1] == finish[1]):
                if(abs(whitePixelsArr[j][0] - finish[0]) - int(robotWidth/2.0) > length):
                    length = abs(whitePixelsArr[j][0] - finish[0]) - int(robotWidth/2.0)
    elif direction == 'down':
        for j in range(len(whitePixelsArr)):
            if(whitePixelsArr[j][0] == finish[0]):
                if(abs(whitePixelsArr[j][1] - finish[1]) - int(robotWidth/2.0) > length):
                    length = abs(whitePixelsArr[j][1] - finish[1]) - int(robotWidth/2.0)
    return length

def getMinLineLength(whitePixelsArr,robotWidth,finish,direction):
    length = robotWidth
    if direction == 'up':
        for j in range(len(whitePixelsArr)):
            if(whitePixelsArr[j][0] == finish[0]):
                if(abs(finish[1] - whitePixelsArr[j][1]) < length):
                    length = abs(finish[1] - whitePixelsArr[j][1])
    elif direction == 'left':
        for j in range(len(whitePixelsArr)):
            if(whitePixelsArr[j][1] == finish[1]):
                if(abs(finish[0] - whitePixelsArr[j][0]) < length):
                    # print("finish[0]: {0}".format(finish[0]))
                    # print("whitePixelsArr[j][0]: {0}".format(whitePixelsArr[j][0]))
                    length = abs(finish[0] - whitePixelsArr[j][0])
    elif direction == 'right':
        for j in range(len(whitePixelsArr)):
            if(whitePixelsArr[j][1] == finish[1]):
                if(abs(whitePixelsArr[j][0] - finish[0]) < length):
                    length = abs(whitePixelsArr[j][0] - finish[0])
    elif direction == 'down':
        for j in range(len(whitePixelsArr)):
            if(whitePixelsArr[j][0] == finish[0]):
                if(abs(whitePixelsArr[j][1] - finish[1]) < length):
                    length = abs(whitePixelsArr[j][1] - finish[1])
    return length

def goUP(whitePixelsArr, updatedImage,robotWidth,finish,markedPixels,fixedLength,firstTimeFlag,returnBack):
    if fixedLength > 0:
        length = fixedLength
    elif fixedLength < 0:
        length = getMaxLineLength(whitePixelsArr,robotWidth,finish,'up') - robotLength
    else:
        length = getMaxLineLength(whitePixelsArr,robotWidth,finish,'up')
    print("length: {0}".format(length))
    cv.line(updatedImage,(finish[0],finish[1]),(finish[0],finish[1]-(length)),(0,0,0),robotWidth,)
    cv.line(updatedImage,(finish[0],finish[1]),(finish[0],finish[1]-(length)),(0,255,0),1)
    commands.append("createLine({0},{1},{2},{3},'front',{4})".format([finish[0]*resolution,(areaHeight-finish[1])*resolution-offset],target,angle,length*resolution,speed))
    finish = [finish[0],finish[1]-(length)]
    if returnBack == True:
        if firstTimeFlag == 1:
            commands.append("createLine({0},{1},{2},-{3},'back',{4})".format([finish[0]*resolution,(areaHeight-finish[1])*resolution-offset],target,angle,(length-int(robotWidth/2.0))*resolution,speed))
            finish = [finish[0],finish[1]+(length-int(robotWidth/2.0))]
            # commands.append("createLine({0},{1},{2},{3},{4},{5})".format(finish,target,angle,length-int(robotWidth/2.0),'back',speed))
        else:
            commands.append("createLine({0},{1},{2},-{3},'back',{4})".format([finish[0]*resolution,(areaHeight-finish[1])*resolution-offset],target,angle,length*resolution,speed))
            finish = [finish[0],finish[1]+(length)]
            # commands.append("createLine({0},{1},{2},{3},{4},{5})".format(finish,target,angle,length,'back',speed))
    cv.imwrite("updatedImage.png",updatedImage)
    updatedImage = cv.imread("updatedImage.png")
    originalImage = cv.imread("inflated.png")
    plt.rcParams["figure.figsize"] = (200,200)
    plt.imshow(updatedImage)
    plt.imshow(updatedImage)
    whitePixelsArr,markedPixels = getWhitePixels(updatedImage,originalImage,corner1,corner2,markedPixels)
    whitePixelsArr.sort(key = lambda whitePixelsArr:(-whitePixelsArr[0],-whitePixelsArr[1]))
    print("finish coordinates after going UP: {}".format(finish))
    return finish, whitePixelsArr,markedPixels

def goDOWN(whitePixelsArr, updatedImage,robotWidth,finish,markedPixels,fixedLength,returnBack):
    if fixedLength > 0:
        length = fixedLength
    elif fixedLength < 0:
        length = getMaxLineLength(whitePixelsArr,robotWidth,finish,'down') - robotLength
    else:
        length = getMaxLineLength(whitePixelsArr,robotWidth,finish,'down')
    print("length: {}".format(length))
    cv.line(updatedImage,(finish[0],finish[1]),(finish[0],finish[1]+(length)),(0,0,0),robotWidth)
    cv.line(updatedImage,(finish[0],finish[1]),(finish[0],finish[1]+(length)),(0,255,0),1)
    commands.append("createLine({0},{1},{2},{3},'front',{4})".format([finish[0]*resolution,(areaHeight-finish[1])*resolution-offset],target,-angle,length*resolution,speed))
    finish = [finish[0],finish[1]+(length)]
    cv.imwrite("updatedImage.png",updatedImage)
    updatedImage = cv.imread("updatedImage.png")
    plt.rcParams["figure.figsize"] = (200,200)
    plt.imshow(updatedImage)
    updatedImage = cv.imread("updatedImage.png")
    originalImage = cv.imread("inflated.png")
    whitePixelsArr,markedPixels = getWhitePixels(updatedImage,originalImage,corner1,corner2,markedPixels)
    whitePixelsArr.sort(key = lambda whitePixelsArr:(-whitePixelsArr[0],-whitePixelsArr[1]))
    print("finish coordinates after going DOWN: {}".format(finish))
    return finish, whitePixelsArr,markedPixels

def goRIGHT(whitePixelsArr, updatedImage,robotWidth,finish,markedPixels,fixedLength,returnBack):
    if fixedLength > 0:
        length = fixedLength
    elif fixedLength < 0:
        length = getMaxLineLength(whitePixelsArr,robotWidth,finish,'right') - robotLength
    else:
        length = getMaxLineLength(whitePixelsArr,robotWidth,finish,'right')
    print("length: {}".format(length))
    cv.line(updatedImage,(finish[0],finish[1]),(finish[0]+(length),finish[1]),(0,0,0),robotWidth)
    cv.line(updatedImage,(finish[0],finish[1]),(finish[0]+(length),finish[1]),(0,255,0),1)
    commands.append("createLine({0},{1},{2},{3},'front',{4})".format([finish[0]*resolution,(areaHeight-finish[1])*resolution-offset],target,angle-sideAngle,length*resolution,speed))
    finish = [finish[0]+(length),finish[1]]
    cv.imwrite("updatedImage.png",updatedImage)
    updatedImage = cv.imread("updatedImage.png")
    plt.rcParams["figure.figsize"] = (200,200)
    plt.imshow(updatedImage)
    updatedImage = cv.imread("updatedImage.png")
    originalImage = cv.imread("inflated.png")
    whitePixelsArr,markedPixels = getWhitePixels(updatedImage,originalImage,corner1,corner2,markedPixels)
    whitePixelsArr.sort(key = lambda whitePixelsArr:(-whitePixelsArr[0],-whitePixelsArr[1]))
    print("finish coordinates after going RIGHT: {}".format(finish))
    return finish, whitePixelsArr,markedPixels

def goLEFT(whitePixelsArr, updatedImage,robotWidth,finish,markedPixels,fixedLength,returnBack):
    if fixedLength > 0:
        length = fixedLength
        # print("A")
    elif fixedLength < 0:
        length = getMaxLineLength(whitePixelsArr,robotWidth,finish,'left') - robotLength
    else:
        length = getMaxLineLength(whitePixelsArr,robotWidth,finish,'left')
        # print("B")
    print("length: {}".format(length))
    cv.line(updatedImage,(finish[0],finish[1]),(finish[0]-(length),finish[1]),(0,0,0),robotWidth)
    cv.line(updatedImage,(finish[0],finish[1]),(finish[0]-(length),finish[1]),(0,255,0),1)
    commands.append("createLine({0},{1},{2},{3},'front',{4})".format([finish[0]*resolution,(areaHeight-finish[1])*resolution-offset],target,angle+sideAngle,length*resolution,speed))
    finish = [finish[0]-(length),finish[1]]
    cv.imwrite("updatedImage.png",updatedImage)
    updatedImage = cv.imread("updatedImage.png")
    plt.rcParams["figure.figsize"] = (200,200)
    plt.imshow(updatedImage)
    updatedImage = cv.imread("updatedImage.png")
    originalImage = cv.imread("inflated.png")
    whitePixelsArr,markedPixels = getWhitePixels(updatedImage,originalImage,corner1,corner2,markedPixels)
    whitePixelsArr.sort(key = lambda whitePixelsArr:(-whitePixelsArr[0],-whitePixelsArr[1]))
    print("finish coordinates after going LEFT: {}".format(finish))
    return finish, whitePixelsArr,markedPixels

def goCURVE(radius,finish,whitePixelsArr,markedPixels,updatedImage,direction):
    if direction == '12-3':
        p0,p1,p2 = BezierPoints([finish[0],finish[1]],[finish[0]+radius,finish[1]],[finish[0]+radius,finish[1]+radius])
        commands.append("bezierByAngle({0},{1},{2},{3},{4},'front',{5})".format([finish[0]*resolution,(areaHeight-finish[1])*resolution-offset],target,0,-angle,radius*resolution,curveSpeed))
        finish = [finish[0]+radius,finish[1]+radius]
        print("finish coordinates after going 12-3: {}".format(finish))
        xy = np.array([B(t,p0,p1,p2) for t in np.linspace(0, 1, 20)]).T
        plt.plot(xy[0],xy[1], color="black",linewidth=robotWidth)
        plt.plot(xy[0],xy[1], color="green",linewidth=10)
    elif direction == '3-12':
        p0,p1,p2 = BezierPoints([finish[0],finish[1]],[finish[0]-radius,finish[1]],[finish[0]-radius,finish[1]-radius])
        commands.append("bezierByAngle({0},{1},{2},{3},{4},'front',{5})".format([finish[0]*resolution,(areaHeight-finish[1])*resolution-offset],target,angle,angle,radius*resolution,curveSpeed))
        finish = [finish[0]-radius,finish[1]-radius]
        print("finish coordinates after going 3-12: {}".format(finish))
        xy = np.array([B(t,p0,p1,p2) for t in np.linspace(0, 1, 20)]).T
        plt.plot(xy[0],xy[1], color="black",linewidth=robotWidth)
        plt.plot(xy[0],xy[1], color="green",linewidth=10)
    elif direction == '3-6':
        p0,p1,p2 = BezierPoints([finish[0],finish[1]],[finish[0],finish[1]+radius],[finish[0]-radius,finish[1]+radius])
        commands.append("bezierByAngle({0},{1},{2},{3},{4},'front',{5})".format([finish[0]*resolution,(areaHeight-finish[1])*resolution-offset],target,-angle,-angle,radius*resolution,curveSpeed))
        finish = [finish[0]-radius,finish[1]+radius]
        print("finish coordinates after going 3-6: {}".format(finish))
        xy = np.array([B(t,p0,p1,p2) for t in np.linspace(0, 1, 20)]).T
        plt.plot(xy[0],xy[1], color="black",linewidth=robotWidth)
        plt.plot(xy[0],xy[1], color="green",linewidth=10)
    elif direction == '6-3':
        p0,p1,p2 = BezierPoints([finish[0],finish[1]],[finish[0],finish[1]-radius],[finish[0]+radius,finish[1]-radius])
        commands.append("bezierByAngle({0},{1},{2},{3},{4},'front',{5})".format([finish[0]*resolution,(areaHeight-finish[1])*resolution-offset],target,0,angle,radius*resolution,curveSpeed))
        finish = [finish[0]+radius,finish[1]-radius]
        print("finish coordinates after going 6-3: {}".format(finish))
        xy = np.array([B(t,p0,p1,p2) for t in np.linspace(0, 1, 20)]).T
        plt.plot(xy[0],xy[1], color="black",linewidth=robotWidth)
        plt.plot(xy[0],xy[1], color="green",linewidth=10)
    elif direction == '6-9':
        p0,p1,p2 = BezierPoints([finish[0],finish[1]],[finish[0]-radius,finish[1]],[finish[0]-radius,finish[1]-radius])
        commands.append("bezierByAngle({0},{1},{2},{3},{4},'front',{5})".format([finish[0]*resolution,(areaHeight-finish[1])*resolution-offset],target,angle+sideAngle,-angle,radius*resolution,curveSpeed))
        finish = [finish[0]-radius,finish[1]-radius]
        print("finish coordinates after going 6-9: {}".format(finish))
        xy = np.array([B(t,p0,p1,p2) for t in np.linspace(0, 1, 20)]).T
        plt.plot(xy[0],xy[1], color="black",linewidth=robotWidth)
        plt.plot(xy[0],xy[1], color="green",linewidth=10)
    elif direction == '9-6':
        p0,p1,p2 = BezierPoints([finish[0],finish[1]],[finish[0],finish[1]+radius],[finish[0]+radius,finish[1]+radius])
        commands.append("bezierByAngle({0},{1},{2},{3},{4},'back',{5})".format([finish[0]*resolution,(areaHeight-finish[1])*resolution-offset],target,-angle,angle,radius*resolution,curveSpeed))
        finish = [finish[0]+radius,finish[1]+radius]
        print("finish coordinates after going 9-6: {}".format(finish))
        xy = np.array([B(t,p0,p1,p2) for t in np.linspace(0, 1, 20)]).T
        plt.plot(xy[0],xy[1], color="black",linewidth=robotWidth)
        plt.plot(xy[0],xy[1], color="green",linewidth=10)
    elif direction == '9-12':
        p0,p1,p2 = BezierPoints([finish[0],finish[1]],[finish[0],finish[1]-radius],[finish[0]+radius,finish[1]-radius])
        commands.append("bezierByAngle({0},{1},{2},{3},{4},'front',{5})".format([finish[0]*resolution,(areaHeight-finish[1])*resolution-offset],target,angle,-angle,radius*resolution,curveSpeed))
        finish = [finish[0]+radius,finish[1]-radius]
        print("finish coordinates after going 9-12: {}".format(finish))
        xy = np.array([B(t,p0,p1,p2) for t in np.linspace(0, 1, 20)]).T
        plt.plot(xy[0],xy[1], color="black",linewidth=robotWidth)
        plt.plot(xy[0],xy[1], color="green",linewidth=10)
    elif direction == '12-9':
        p0,p1,p2 = BezierPoints([finish[0],finish[1]],[finish[0]-radius,finish[1]],[finish[0]-radius,finish[1]+radius])
        commands.append("bezierByAngle({0},{1},{2},{3},{4},'front',{5})".format([finish[0]*resolution,(areaHeight-finish[1])*resolution-offset],target,angle+sideAngle,angle,radius*resolution,curveSpeed))
        finish = [finish[0]-radius,finish[1]+radius]
        print("finish coordinates after going 12-9: {}".format(finish))
        xy = np.array([B(t,p0,p1,p2) for t in np.linspace(0, 1, 20)]).T
        plt.plot(xy[0],xy[1], color="black",linewidth=robotWidth)
        plt.plot(xy[0],xy[1], color="green",linewidth=10)
    cv.imwrite("updatedImage.png",updatedImage)
    updatedImage = cv.imread("updatedImage.png")
    plt.rcParams["figure.figsize"] = (200,200)
    plt.imshow(updatedImage)
    updatedImage = cv.imread("updatedImage.png")
    originalImage = cv.imread("inflated.png")
    whitePixelsArr,markedPixels = getWhitePixels(updatedImage,originalImage,corner1,corner2,markedPixels)
    whitePixelsArr.sort(key = lambda whitePixelsArr:(-whitePixelsArr[0],-whitePixelsArr[1]))
    return finish, whitePixelsArr,markedPixels

def verticalLine(whitePixelsArr, updatedImage,robotWidth,finish,markedPixels,firstTimeFlag,returnBack):
    # length = 0
    finish, whitePixelsArr,markedPixels = goUP(whitePixelsArr,updatedImage,robotWidth,finish,markedPixels,-robotLength,firstTimeFlag,returnBack=True)
    return finish, whitePixelsArr

def verticalLineCurve(whitePixelsArr,updatedImage,robotWidth,finish,markedPixels,direction):
    if direction == 'up':
        finish, whitePixelsArr,markedPixels = goCURVE(int(robotWidth/2.0),finish,whitePixelsArr,markedPixels,updatedImage,direction='6-9')
    elif direction == 'down':
        finish, whitePixelsArr,markedPixels = goCURVE(int(robotWidth/2.0),finish,whitePixelsArr,markedPixels,updatedImage,direction='9-6')
    return finish, whitePixelsArr

def goAround(whitePixelsArr,robotWidth,finish,updatedImage,markedPixels):
    flag = True
    while flag == True:
        # print("finish: {0}".format(finish))
        # print(whitePixelsArr)
        
        flag = False

        if [finish[0] - int(robotWidth/2.0+1),finish[1]] in whitePixelsArr:
            flag = True
            length = getMaxLineLength(whitePixelsArr,robotWidth,finish,direction='left')
            finish, whitePixelsArr,markedPixels = goLEFT(whitePixelsArr,updatedImage,robotWidth,finish,markedPixels,-robotLength,returnBack=False)
            finish, whitePixelsArr,markedPixels = goCURVE(int(robotWidth/2.0),finish,whitePixelsArr,markedPixels,updatedImage,direction='6-9')
            # print("if1 length: {0}".format(length))
        if [finish[0],finish[1] - int(robotWidth/2.0+1)] in whitePixelsArr:
            flag = True
            length = getMaxLineLength(whitePixelsArr,robotWidth,finish,direction='up')
            finish, whitePixelsArr,markedPixels = goUP(whitePixelsArr,updatedImage,robotWidth,finish,markedPixels,-robotLength,0,returnBack=False)
            finish, whitePixelsArr,markedPixels = goCURVE(int(robotWidth/2.0),finish,whitePixelsArr,markedPixels,updatedImage,direction='9-12')
            # print("if2 length: {0}".format(length))
        if [finish[0] + int(robotWidth/2.0+1),finish[1]] in whitePixelsArr:
            flag = True
            length = getMaxLineLength(whitePixelsArr,robotWidth,finish,direction='right')
            finish, whitePixelsArr,markedPixels = goRIGHT(whitePixelsArr,updatedImage,robotWidth,finish,markedPixels,-robotLength,returnBack=False)
            finish, whitePixelsArr,markedPixels = goCURVE(int(robotWidth/2.0),finish,whitePixelsArr,markedPixels,updatedImage,direction='12-3')
            # print("if3 length: {0}".format(length))
        if [finish[0],finish[1] + int(robotWidth/2.0+1)] in whitePixelsArr:
            flag = True
            length = getMaxLineLength(whitePixelsArr,robotWidth,finish,direction='down')
            finish, whitePixelsArr,markedPixels = goDOWN(whitePixelsArr,updatedImage,robotWidth,finish,markedPixels,-robotLength,returnBack=False)
            finish, whitePixelsArr,markedPixels = goCURVE(int(robotWidth/2.0),finish,whitePixelsArr,markedPixels,updatedImage,direction='3-6')
            # print("if4 length: {0}".format(length))
        print("Going around checkpoints: {0}".format(finish))
    return
    
def main(whitePixelsArr,robotWidth,updatedImage,markedPixels):
    flag = True
    finish = [whitePixelsArr[0][0],whitePixelsArr[0][1]]
    firstTimeFlag = 1
    while flag == True:
        # print(markedPixels)
        flag = False
        # print("status before: {0}".format(flag))
        # print("starting coordinates: {0}".format(finish))
        if firstTimeFlag == 1:
            finish, whitePixelsArr = verticalLine(whitePixelsArr,updatedImage,robotWidth,finish,markedPixels,firstTimeFlag,returnBack=True)
            finish, whitePixelsArr = verticalLineCurve(whitePixelsArr,updatedImage,robotWidth,finish,markedPixels,direction='down')
            finish, whitePixelsArr,markedPixels = goLEFT(whitePixelsArr,updatedImage,robotWidth,finish,markedPixels,int(robotWidth/2.0),returnBack=True)
            firstTimeFlag = 0
            flag = True
        else:
            # if [finish[0],finish[1]-int(robotWidth/2.0+1)] in whitePixelsArr:
            #     finish, whitePixelsArr = verticalLineCurve(whitePixelsArr,updatedImage,robotWidth,finish,markedPixels,direction='up')
            #     finish, whitePixelsArr = verticalLine(whitePixelsArr,updatedImage,robotWidth,finish,markedPixels,firstTimeFlag,returnBack=True)
            #     finish, whitePixelsArr = verticalLineCurve(whitePixelsArr,updatedImage,robotWidth,finish,markedPixels,direction='down')
            #     flag = True
            #     print("if")
            if [finish[0]-int(robotWidth/2.0),finish[1]-int(robotWidth/2.0)] in whitePixelsArr:
                finish, whitePixelsArr = verticalLineCurve(whitePixelsArr,updatedImage,robotWidth,finish,markedPixels,direction='up')
                finish, whitePixelsArr = verticalLine(whitePixelsArr,updatedImage,robotWidth,finish,markedPixels,firstTimeFlag,returnBack=True)
                finish, whitePixelsArr = verticalLineCurve(whitePixelsArr,updatedImage,robotWidth,finish,markedPixels,direction='down')
                finish, whitePixelsArr,markedPixels = goLEFT(whitePixelsArr,updatedImage,robotWidth,finish,markedPixels,int(robotWidth/2.0),returnBack=True)
                flag = True
                # print("elif")
            elif [finish[0]-int(robotWidth+1),finish[1]] in whitePixelsArr:
                flag = True
                length = getMinLineLength(whitePixelsArr,robotWidth,finish,direction='left')
                # print("min length: {0}".format(length))
                finish, whitePixelsArr,markedPixels = goLEFT(whitePixelsArr,updatedImage,robotWidth,finish,markedPixels,length,returnBack=True)
                # finish = [finish[0]-1,finish[1]]
                # print("else")

    return finish, whitePixelsArr,markedPixels,updatedImage

In [11]:
#main


inflated = prepareImage("warehouse1.png",imageAngle)
original = cv.imread("warehouse1.png")
offset = calculateOffset(original,imageAngle)
print("offset: {0}".format(offset))
cv.rectangle(inflated,(corner1[0]-1,corner1[1]+1),(corner2[0]+1,corner2[1]-1),(0,0,255),1)
cv.rectangle(inflated,(corner1[0]+(int(robotWidth/2.0)-1),corner1[1]-(int(robotWidth/2.0)-1)),(corner2[0]-(int(robotWidth/2.0)-1),corner2[1]+(int(robotWidth/2.0)-1)),(0,0,255),1)
cv.imwrite("warehouseInflated.png",inflated)
cv.imwrite("updatedImage.png",inflated)

markedPixels = []
whitePixelsArr,markedPixels = getWhitePixels(inflated, inflated,corner1, corner2,markedPixels)

whitePixelsArr.sort(key = lambda whitePixelsArr:(-whitePixelsArr[0],-whitePixelsArr[1]))
size = inflated.shape
areaWidth = size[1]
areaHeight = size[0]
# updatedImage,flag,markedPixels = findNewPixels(robotWidth,whitePixelsArr,inflated)
finish = whitePixelsArr[0]
print(finish)
finish,whitePixelsArr,markedPixels,updatedImage = main(whitePixelsArr,robotWidth,inflated,markedPixels)
print("Going around")
# print(markedPixels)
whitePixelsArr,markedPixels = getWhitePixels(inflated, inflated,corner1, corner2,markedPixels)
whitePixelsArr.sort(key = lambda whitePixelsArr:(-whitePixelsArr[0],-whitePixelsArr[1]))
goAround(whitePixelsArr,robotWidth,finish,inflated,markedPixels)
# for i in range(len(commands)):
#     commands[i][0] = [commands[i][0][0]*resolution,commands[i][0][0]*resolution]
print(commands)
inflated = cv.imread("updatedImage.png")
cv.rectangle(inflated,(corner1[0]-1,corner1[1]+1),(corner2[0]+1,corner2[1]-1),(0,0,255),1)
cv.rectangle(inflated,(corner1[0]+(int(robotWidth/2.0)-1),corner1[1]-(int(robotWidth/2.0)-1)),(corner2[0]-(int(robotWidth/2.0)-1),corner2[1]+(int(robotWidth/2.0)-1)),(0,0,255),1)
print("Done")

# cv.circle(inflated,(709,55), 1, (0,0,255),-1)

img = mpimg.imread('updatedImage.png')
plt.rcParams["figure.figsize"] = (500,400)
plt.imshow(img)
cv.waitKey(0)
cv.destroyAllWindows()

2474
offset: 7.569739179334205
[1625, 1275]
length: 129
finish coordinates after going UP: [1625, 1250]
finish coordinates after going 9-6: [1650, 1275]
length: 25
finish coordinates after going LEFT: [1625, 1275]
length: 26
finish coordinates after going LEFT: [1599, 1275]
finish coordinates after going 6-9: [1574, 1250]
length: 125
finish coordinates after going UP: [1574, 1250]
finish coordinates after going 9-6: [1599, 1275]
length: 25
finish coordinates after going LEFT: [1574, 1275]
length: 26
finish coordinates after going LEFT: [1548, 1275]
finish coordinates after going 6-9: [1523, 1250]
length: 125
finish coordinates after going UP: [1523, 1250]
finish coordinates after going 9-6: [1548, 1275]
length: 25
finish coordinates after going LEFT: [1523, 1275]
length: 26
finish coordinates after going LEFT: [1497, 1275]
finish coordinates after going 6-9: [1472, 1250]
length: 125
finish coordinates after going UP: [1472, 1250]
finish coordinates after going 9-6: [1497, 1275]
length: